In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import metrics

In [10]:
# load the data set after data preprocessing
train=pd.read_csv('train_set_aftersplit.csv', index_col=0)
test=pd.read_csv('test_set_aftersplit.csv', index_col = 0)
train_x=train.iloc[:,:-1]
train_y=train.iloc[:,-1]
test_x=test.iloc[:,:-1]
test_y=test.iloc[:,-1]
test_y_vad=list(test_y)

In [11]:
# the score of cross validation
def score_cv(model,X,y):
    kfold= KFold(n_splits=5,random_state=42,shuffle=True)
    f1=cross_val_score(model,X,y,scoring='f1_macro',cv=kfold).mean()
    return f1

# grid research
def gridsearch_cv(model,test_param,cv=5):
    gridsearch=GridSearchCV(estimator=model,param_grid=test_param,scoring='f1_macro',n_jobs=-1,cv=cv)
    gridsearch.fit(train_x,train_y)
    print('Best Params: ',gridsearch.best_params_)
    print('Best Score: ',gridsearch.best_score_)
    return gridsearch.best_params_

# model evaluation
def model_evaluation(pred_res,true_res):
    aa=ag=au=ga=gg=gu=ua=ug=uu=0
    for i in range(0,len(pred_res)):
        if pred_res[i]=='acc'and true_res[i]=='acc':
            aa+=1
        elif pred_res[i]=='acc'and true_res[i]=='good':
            ag+=1
        elif pred_res[i]=='acc'and true_res[i]=='unacc':
            au+=1
        elif pred_res[i]=='unacc'and true_res[i]=='acc':
            ua+=1
        elif pred_res[i]=='unacc'and true_res[i]=='unacc':
            uu+=1
        elif pred_res[i]=='unacc'and true_res[i]=='good':
            ug+=1
        elif pred_res[i]=='good'and true_res[i]=='acc':
            ga+=1
        elif pred_res[i]=='good'and true_res[i]=='unacc':
            gu+=1
        elif pred_res[i]=='good'and true_res[i]=='good':
            gg+=1
    # f_score for 'acc' as true value
    accuracy1=aa/(aa+ga+ua)
    precision1=aa/(aa+ag+au)
    recall1=aa/(aa+ga+ua)
    f_measure1=2*(precision1*recall1)/(precision1+recall1)
    acc_score={'accuracy': accuracy1, 'precision': precision1, 'recall': recall1, 'f_measure': f_measure1}
    print('acc_score:{}'.format(acc_score))
    
    # f_score for 'unacc' as true value
    accuracy2=uu/(uu+au+gu)
    precision2=uu/(uu+ua+ug)
    recall2=uu/(uu+au+gu)
    f_measure2=2*(precision2*recall2)/(precision2+recall2)
    unacc_score={'accuracy': accuracy2, 'precision': precision2, 'recall': recall2, 'f_measure': f_measure2}
    print('unacc_score:{}'.format(unacc_score))
    
    # f_score for 'good' as true value
    accuracy3=gg/(gg+ag+ug)
    try:
        precision3=gg/(ga+gg+gu)
    except:
        precision3 = 0
    try:
        recall3=gg/(gg+ag+ug)
    except:
        recall3 = 0
    try:
        f_measure3=2*(precision3*recall3)/(precision3+recall3)
    except:
        f_measure3 = 0
    good_score={'accuracy': accuracy3, 'precision': precision3, 'recall': recall3, 'f_measure': f_measure3}
    print('good_score:{}'.format(good_score))
    sum_accuracy=(aa+gg+uu)/(aa+ag+au+ga+gg+gu+ua+ug+uu)
    macro_precision=(precision1+precision2+precision3)/3
    macro_recall=(recall1+recall2+recall3)/3
    macro_f1=(f_measure1+f_measure2+f_measure3)/3
    sum_score={'accuracy': sum_accuracy, 'macro_precision': macro_precision, 'macro_recall': macro_recall, 'macro_f1': macro_f1}
    print('macro_score:{}'.format(sum_score))

In [12]:
def Perceptron(train_x,train_y,test_x, iterations, learning_rate):
    train_y1=train_y
    train_y=train_y.map({'good':0,'acc':0,'unacc':1})
    train_y1=train_y1.map({'good':0,'acc':1})
    train_y=list(train_y)
    train_y1=list(train_y1)
    count=0
    # initialize the weight randomly 
    weight=[np.random.rand()]*21
    bias=0
    weight1=[np.random.rand()]*21
    bias1=0
    while(count<iterations):
        flag=0
        for i in range(0,len(train_x)-2):
            val=bias+(weight*(train_x.loc[i,:])).sum()
            if(val>0):
                if train_y[i]-1!=0:
                    flag=1
                bias=bias+learning_rate*(train_y[i]-1)
                weight=weight+learning_rate*(train_y[i]-1)*train_x.loc[i,:]
            else:
                if train_y[i]-0!=0:
                    flag=1
                    bias=bias+learning_rate*(train_y[i])
                    weight=weight+learning_rate*(train_y[i])*train_x.loc[i,:]
                else:
                    val_1=bias1+(weight1*train_x.loc[i,:]).sum()
                    if val_1>0:
                        if train_y1[i]-1!=0:
                            flag=1
                        bias1=bias1+learning_rate*(train_y1[i]-1)
                        weight1=weight1+learning_rate*(train_y1[i]-1)*train_x.loc[i,:]
                    else:
                        if train_y1[i]-0!=0:
                            flag=1
                        bias1=bias1+learning_rate*train_y1[i]
                        weight1=weight1+learning_rate*(train_y1[i])*train_x.loc[i,:]
        if flag==0:
            print('After {} iterations, it has converged!'.format(count))
            break
        print('{}/{} finished'.format(count+1, iterations))
        count+=1
        
    #Get the predict result
    pred_res=[]
    for i in range(0,len(test_x)-1):
        v=bias+(weight*test_x.loc[i,:]).sum()
        if v>0:
            pred_res.append('unacc')
        else:
            v1=bias1+(weight1*test_x.loc[i,:]).sum()
            if v1<0:
                pred_res.append('good')
            else:
                pred_res.append('acc')
    return pred_res

In [13]:
train_x=train_x.reset_index(drop=True)
test_x=test_x.reset_index(drop=True)

In [14]:
pred_res=Perceptron(train_x,train_y,test_x,50,0.2)

1/50 finished
2/50 finished
3/50 finished
4/50 finished
5/50 finished
6/50 finished
7/50 finished
8/50 finished
9/50 finished
10/50 finished
11/50 finished
12/50 finished
13/50 finished
14/50 finished
15/50 finished
16/50 finished
17/50 finished
18/50 finished
19/50 finished
20/50 finished
21/50 finished
22/50 finished
23/50 finished
24/50 finished
25/50 finished
26/50 finished
27/50 finished
28/50 finished
29/50 finished
30/50 finished
31/50 finished
32/50 finished
33/50 finished
34/50 finished
35/50 finished
36/50 finished
37/50 finished
38/50 finished
39/50 finished
40/50 finished
41/50 finished
42/50 finished
43/50 finished
44/50 finished
45/50 finished
46/50 finished
47/50 finished
48/50 finished
49/50 finished
50/50 finished


In [15]:
model_evaluation(pred_res,test_y_vad)

acc_score:{'accuracy': 0.8648648648648649, 'precision': 0.7619047619047619, 'recall': 0.8648648648648649, 'f_measure': 0.810126582278481}
unacc_score:{'accuracy': 0.9224489795918367, 'precision': 0.9741379310344828, 'recall': 0.9224489795918367, 'f_measure': 0.9475890985324947}
good_score:{'accuracy': 0.9166666666666666, 'precision': 0.7333333333333333, 'recall': 0.9166666666666666, 'f_measure': 0.8148148148148148}
macro_score:{'accuracy': 0.9093655589123867, 'macro_precision': 0.8231253420908593, 'macro_recall': 0.9013268370411227, 'macro_f1': 0.8575101652085969}
